In [17]:
import keras
import tensorflow as tf
import numpy as np
import matplotlib as plt
import keras.layers as layers

In [18]:
class  CustomerTicketModel(keras.Model):
    def __init__(self, num_departments):
        super().__init__()
        self.concat_layer = layers.Concatenate()
        self.mixing_layer = layers.Dense(64, activation="relu")
        self.priority_scorer = layers.Dense(1, activation="sigmoid")
        self.department_classifier = layers.Dense(num_departments, activation="softmax")

    def call(self, inputs):
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]
        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)
        department = self.department_classifier(features)
        return priority, department
        

In [19]:

num_samples = 1280
vocabulary_size = 10000
num_tags = 10
num_departments = 4

title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))
print(title_data.shape)
print(text_body_data.shape)
print(tags_data.shape)


priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))


(1280, 10000)
(1280, 10000)
(1280, 10)


In [20]:
model = CustomerTicketModel(num_departments=4)

In [25]:
model.compile(
    optimizer="rmsprop",
    loss=["mean_squared_error", "categorical_crossentropy"],
    metrics=[["mean_absolute_error"], ["accuracy"]]
)

tf_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs_sub_model")

model.fit({
    "title": title_data,
    "tags": tags_data,
    "text_body": text_body_data
    },
    [priority_data, department_data],
    epochs=10,
    callbacks=[tf_callback]
    )

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2793 - loss: 39.9102 - mean_absolute_error: 0.5017 
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2533 - loss: 33.8154 - mean_absolute_error: 0.5101
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2525 - loss: 44.5175 - mean_absolute_error: 0.4931
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2627 - loss: 53.1748 - mean_absolute_error: 0.5069
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2454 - loss: 64.5052 - mean_absolute_error: 0.5001
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2656 - loss: 69.5198 - mean_absolute_error: 0.5027
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2791 - loss: 76.8951 - mean_absolute_error: 0.5002
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2799 - loss: 89.3041 - mean_absolute_error: 0.5059
Epoch 9/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3122 - loss: 97

In [29]:
model.evaluate(
    {
        "title":title_data,
        "text_body":text_body_data,
        "tags":tags_data
    },
    [priority_data, department_data]
)

priority, department = model({"title": title_data,
                             "text_body": text_body_data,
                             "tags": tags_data})
print("priority",priority)
print("department",department)
model.summary()

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4081 - loss: 92.8033 - mean_absolute_error: 0.4980
priority tf.Tensor(
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]], shape=(1280, 1), dtype=float32)
department tf.Tensor(
[[1.6694687e-01 0.0000000e+00 8.3305317e-01 9.4257653e-24]
 [4.6818228e-03 0.0000000e+00 9.9531811e-01 1.3664090e-23]
 [2.4393630e-04 0.0000000e+00 9.9975604e-01 9.5471001e-22]
 ...
 [7.9534042e-01 0.0000000e+00 2.0465964e-01 3.7160178e-22]
 [2.6834395e-03 0.0000000e+00 9.9731654e-01 1.6672225e-25]
 [4.7551999e-03 0.0000000e+00 9.9524480e-01 5.8114252e-20]], shape=(1280, 4), dtype=float32)


Model: "customer_ticket_model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ concatenate_2 (Concatenate)     │ (1280, 20010)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (1280, 64)             │     1,280,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (1280, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (1280, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,562,060 (9.77 MB)

 Trainable params: 1,281,029 (4.89 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,281,031 (4.89 MB)

In [28]:

priority_preds, department_preds = model.predict({"title": title_data,                                                  "text_body": text_body_data,
                                                  "tags": tags_data})
print("priority_preds", priority_preds)
print("demartment_preds", department_preds)

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
priority_preds [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
demartment_preds [[1.6640429e-01 0.0000000e+00 8.3359575e-01 9.4135008e-24]
 [4.6727303e-03 0.0000000e+00 9.9532723e-01 1.3637553e-23]
 [2.4393630e-04 0.0000000e+00 9.9975604e-01 9.5471001e-22]
 ...
 [7.9565805e-01 0.0000000e+00 2.0434190e-01 3.7102484e-22]
 [2.6834395e-03 0.0000000e+00 9.9731654e-01 1.6672225e-25]
 [4.7551999e-03 0.0000000e+00 9.9524480e-01 5.8114252e-20]]
